
<div id="navegacion" prev="../Ejercicios_Tema_7/ej6.html" next=""></div>

<div class="section-title">Sección 6.5</div>

#  Movilens 1M


Movilens Dataset recoge datos de 6000 usuarios y 4000  películas. La información se distribuye en 3 tablas.

- [ratings.dat](./ml-1m/ratings.dat) - El fichero recoge datos de valoraciones  de los usuarios. Las columnas son: Usuario, MovieID, Puntuación, Fecha.

- [movies.dat](./ml-1m/movies.dat) - Datos de las películas. Las columnas son: MovieID, Titulo, Género
- [users.dat](./ml-1m/users.dat) - Datos personales de los usuarios. Las columnas son: Usuario, Sexo, Edad, Ocupación, ZipCode.


Toda esta información se utiliza con mucha frecuencia en los sistemas de recomendación que aplican algoritmos de aprendizaje.
La información del contenido de los ficheros la puedes encontrar en el enlace [README](./ml-1m/README).


## Importar datos

Crear un DataFrame para cada uno de los ficheros.  Usar la función `head()` para visualizar los primeros datos.
El fichero ratings tiene un campo de tipo fecha. Usa el parámetro `parse_dates` para cargar de forma adecuada las fechas.

In [1]:
# Sol:
import pandas as pd

colum_name = ["id", "genero", "año", "ocupacion", "zipcode"]
usuarios = pd.read_csv("users.dat", sep=":", names =colum_name)
usuarios.head(5)

,id,genero,año,ocupacion,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [2]:
colum_name = ["id", "titulo", "tipo"]
peliculas = pd.read_csv("movies.dat", sep=':', names=colum_name)
peliculas.head(5)

,id,titulo,tipo
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
colum_name = ["uid", "pid", "valoracion", "fecha"]
valoraciones = pd.read_csv("ratings.dat", sep= ":", parse_dates= [3], names=colum_name)
valoraciones.head(5)

,uid,pid,valoracion,fecha
0,1139,377,4,2000-11-22 08:29:17
1,1524,3857,4,2000-11-20 19:57:23
2,4823,2987,3,2000-07-09 00:25:18
3,336,2105,5,2000-12-09 12:38:32
4,5131,1752,2,2000-06-27 20:32:25


In [14]:
(valoraciones["fecha"]).dtype

dtype('<M8[ns]')

## Valoraciones de X-Men

Los usuarios valoran las películas con valores del 1 al 5 donde el 1 es la peor puntuación y 5 es la mejor. La puntuación es una variable categórica. Calcula el número de votos de cada tipo (la frecuencia de votos) de la película con título `X-Men (2000)`. 



In [15]:
# Sol:

#Consultando id_peli de x-men(2000)
pos = list(peliculas["titulo"]).index("X-Men (2000)")
pid_x_men = peliculas["id"][pos]
pid_x_men

#Filtrado la lista de valoraciones segun id de x-men, obviamente los que no sale en conteo tendía una frecuencia 0
filtrado = valoraciones[valoraciones["pid"] == pid_x_men]
filtrado.valoracion.value_counts()

4    4
5    3
3    2
2    1
Name: valoracion, dtype: int64

## Puntuación media

Calcula la puntuación media  de cada película por sexo del usuario. Diseña una función llamada `puntuacion_media` que devuelve un Dataframe donde el índice es el título de la película y las columnaa son 'F' y 'M' (Female y Male respectivamente).

__Nota__:

Hay varias formas de hacerlo. Aquí os dejo alguna sugerencia:
* Necesitamos combinar las tres tablas ya que tenemos, por un lado las valoraciones, por otro el título de la película y por otro el sexo de los usuarios. La operación `merge` te permitirá realizar la  combinación.
    * Combina primero las valoraciones con las peliculas por el campo 'id_peli'.
    * Combina el resultado anterior con los usuarios por el campo 'id_usuario'.
    
* Una vez combinadas las tres tablas en una sola tabla:
    * solución 1:  Agrupar por `id_peli` y por `sexo` del usuario. La función de agregación es la media `mean()`. Para obtener la tabla pedida utiliza `unstack` para deshacer el multi índice.
    * solución 2: Consulta la ayuda de la función `pivot_table`. Utiliza la función `pivot_table`. Merece la pena que intentéis resolverlo también utilizando esta opción.


In [29]:
# Sol:
#Suponiendo variables globales y que la función no recibe parámetros
def puntuacion_media():
    """Calcula la puntuación media  de cada película por sexo del usuario
    
    Example:
        >>> puntuacion_media()
               
        genero_u                            F          M
        titulo 
        $1,000,000 Duck (1971)       3.375000    2.761905
        'Night Mother (1986)         3.388889    3.352941
        'Til There Was You (1997)    2.675676    2.733333

        ...
        ...
    
"""
    # Merge de 3 tablas
    mez = valoraciones.merge(peliculas, left_on= "pid", right_on= "id")
    mez = mez.merge(usuarios, left_on= "uid", right_on= "id")
    # Mean y unstack
    result = mez.groupby(by=["titulo", "genero"]).mean().unstack(level= ["genero"], fill_value=0.0)["valoracion"]
    print(result)

puntuacion_media()

genero                                   F         M
titulo                                              
$1,000,000 Duck (1971)                 3.0  0.000000
'Til There Was You (1997)              0.0  1.000000
'burbs, The (1989)                     1.0  2.000000
...And Justice for All (1979)          0.0  4.333333
10 Things I Hate About You (1999)      4.2  4.500000
...                                    ...       ...
Young Poisoner's Handbook, The (1995)  0.0  3.000000
Young Sherlock Holmes (1985)           0.0  3.666667
Your Friends and Neighbors (1998)      0.0  4.000000
Zed & Two Noughts, A (1985)            0.0  1.000000
eXistenZ (1999)                        3.0  2.000000

[2301 rows x 2 columns]


In [22]:
def puntuacion_media_2():
    """Calcula la puntuación media  de cada película por sexo del usuario
    
    Example:
        >>> puntuacion_media()
               
        genero_u                            F          M
        titulo 
        $1,000,000 Duck (1971)       3.375000    2.761905
        'Night Mother (1986)         3.388889    3.352941
        'Til There Was You (1997)    2.675676    2.733333

        ...
        ...
    
"""
    #Merge de 3 tablas
    mez = valoraciones.merge(peliculas, left_on= "pid", right_on= "id")
    mez = mez.merge(usuarios, left_on= "uid", right_on= "id")
    #Pivot_table
    table = pd.pivot_table(mez, values="valoracion", index=["titulo"], columns="genero", fill_value=0.0)
    print(table)

puntuacion_media_2()

genero                                   F         M
titulo                                              
$1,000,000 Duck (1971)                 3.0  0.000000
'Til There Was You (1997)              0.0  1.000000
'burbs, The (1989)                     1.0  2.000000
...And Justice for All (1979)          0.0  4.333333
10 Things I Hate About You (1999)      4.2  4.500000
...                                    ...       ...
Young Poisoner's Handbook, The (1995)  0.0  3.000000
Young Sherlock Holmes (1985)           0.0  3.666667
Your Friends and Neighbors (1998)      0.0  4.000000
Zed & Two Noughts, A (1985)            0.0  1.000000
eXistenZ (1999)                        3.0  2.000000

[2301 rows x 2 columns]
